## 0. Load and prepare Data

In [1]:
import pandas as pd
import numpy as np
from numpy import log
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('baseball.dat', sep=r"\s+")

df.head()

,salary,average,obp,runs,hits,doubles,triples,homeruns,rbis,walks,...,rbisperso,walksperso,obppererror,runspererror,hitspererror,hrspererror,soserrors,sbsobp,sbsruns,sbshits
0,3300,0.272,0.302,69,153,21,4,31,104,22,...,1.3000,0.2750,0.0755,17.2500,38.2500,7.7500,320,1.208,276,612
1,2600,0.269,0.335,58,111,17,2,18,66,39,...,0.9565,0.5652,0.0838,14.5000,27.7500,4.5000,276,0.000,0,0
2,2500,0.249,0.337,54,115,15,1,17,73,63,...,0.6293,0.5431,0.0562,9.0000,19.1667,2.8333,696,2.022,324,690
3,2475,0.260,0.292,59,128,22,7,12,50,23,...,0.7812,0.3594,0.0133,2.6818,5.8182,0.5455,1408,6.132,1239,2688
4,2313,0.273,0.346,87,169,28,5,8,58,70,...,1.0943,1.3208,0.0384,9.6667,18.7778,0.8889,477,1.038,261,507


In [3]:
y_raw = df['salary'].values

# Transform y using natural log because of skewness
y = np.log(y_raw) 


X = df.loc[:, df.columns != 'salary'].values

In [4]:
n = X.shape[0] # number of observations (rows)
p = X.shape[1] # number of predictors (columns) (27 in this case)
print(f"Number of observations: {n}, Number of predictors: {p}")

Number of observations: 337, Number of predictors: 27


In [5]:
# Precompute total sum of squares for y
mean_y = np.mean(y)

# TSS = sum over i=1..n of (y[i] - mean_y)^2
TSS = np.sum((y - mean_y) ** 2)

## 1. Set GA hyperparameters (fixed for now)

In [11]:
P = 20 # Population size
G = 50 # Number of generations
K = 5 # Folds in cross validation
mutation_rate = 0.01 # Mutation rate
seed = 42 # Random seed for reproducibility

## 2. Initialize Population (Generation 0)

In [7]:
# Population = array/list of length P, each element is a binary vector of length p indicating selected predictors
# fitness = array of length P, to store fitness (1 - CV R²) for each individual in the population

# Set random seed for reproducibility
np.random.seed(seed)

# Initialize population: P chromosomes, each of length P (20)
# Values are 0 or 1 with equal probability
# population is a matrix of shape (P, p), where P = population size (20) and p = number of predictors (27)
# each row of population is one chromosome
population = (np.random.rand(P, p) < 0.5).astype(int)

print("Population shape:", population.shape)
print("First chomosome:", population[0])

Population shape: (20, 27)
First chomosome: [1 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1]


## 3. Evaluate Fitness for Generation 0 

Fitness = K-fold CV R²

In [8]:
# fitness number for each chromosome = how good that model is at predicting unseen salaries
# for each chromosome, do K-fold Cross Validation
    # split the data into K folds (e.g. 5) of row indices
    # for each fold:
        # train on K-1 folds
        # predict on the left-out fold
        # compute squared prediction errors
    # Add all squared prediction errors across all folds to get SSPE
    # We already have TSS computed
    # Compute: R²_cv = 1 - (SSPE / TSS)
    # that R²_cv is the fitness for that chromosome
# We do for all 20 chromosomes in the population -> we get fitness array of length 20


# Create KFold splitter ONCE (we'll reuse its pattern for all chromosomes)
# k-fold is a utility from sklearn that generates train/test indices for cross-validation
# n-splits=K means we want K folds
# shuffle=True means we shuffle the data before splitting into folds
# random_state=seed ensures reproducibility
# Important: Kfold only cares about the number of rows, not columns, so using X_sel or X is the same here
kf = KFold(n_splits=K, shuffle=True, random_state=seed)

# Initialize fitness array
fitness = np.zeros(P)

for i in range(P):
    # Chromosome i: a 0/1 vector of length p
    # population[i, :] is the i-th row, a 1-D array of length p
    # b is the chromosome indicating which predictors are selected
    b = population[i, :] # shape (p,)

    # If chromosome selects no predictors, give terrible fitness
    # b.sum() adds up all the 0/1 entries
    # If sum is 0, it means no predictors were selected, a model with no predictors is useless and we give it a very bad fitness, so GA will kill it off
    # continue jumps to the next i without running CV
    if b.sum() == 0:
        fitness[i] = -1e9
        continue

    # Select the columns of X where b[j] == 1
    # b == 1 returns a boolean mask of length p
    # X[:, b == 1]
        # : for rows -> all rows
        # b == 1 for columns -> only columns where b[j] is True (1)
    # if b has 10 ones, X_sel is n * 10
    # this is how we turn the binary chromo`some into a design matrix for regression
    X_sel = X[:, b == 1] # matrix with shape (n, num_selected_predictors)

    SSPE = 0.0 # Sum of squared prediction errors across all folds

    # Cross validation loop
    # Loops over k folds
    # train_index and test_index are row indices, they work regardless of how many columns X_sel has
    # kf.split(X_sel) returns a generator of (train_index, test_index) pairs 
    # for each fold: 
        # train_index: indices of rows for training
        # test_index: indices of rows for testing
    for train_index, test_index in kf.split(X_sel):
        X_train = X_sel[train_index, :]
        y_train = y[train_index]
        X_test = X_sel[test_index, :]
        y_test = y[test_index]
        # X_train: subset of rows of X_sel for training
        # y_train: corresponding subset of y for training
        # X_test, y_test: the same for testing

        # Fit simple linear regression on training data
        # .fit(X_train, y_train) estimates the regression coefficients
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predict on test data
        # y_pred: predicted log-salaries for test set
        y_pred = model.predict(X_test)

        # Accumulate squared prediction error
        # errors: difference between actual and predicted log-salaries
        # np.sum(errors ** 2): sum of squared prediction errors for this fold
        errors = y_test - y_pred
        SSPE += np.sum(errors ** 2) # Acumulates squared projection errors across all folds

    # Compute R² for this chromosome
    # SSPE / TSS = proportion of variance in y not explained by the model
    # 1 - (SSPE / TSS) = proportion of variance in y explained by predictions on unseen data
    R2_cv = 1 - (SSPE / TSS)
    fitness[i] = R2_cv


# After this loop, 'fitness' holds the Generation 0 fitness for all P individuals
print("Generation 0 fitness values:")
print(fitness)
print("Best fitness in Gen 0:", fitness.max())
print("Index of best individual:", fitness.argmax())
print("Best chromosome in Gen 0:", population[fitness.argmax()])

Generation 0 fitness values:
[0.49385646 0.77251873 0.60717637 0.46302315 0.46510764 0.50044643
 0.46657592 0.74180172 0.44833037 0.45795769 0.69440392 0.7601852
 0.44502875 0.46852341 0.45466271 0.60983828 0.4582458  0.45970474
 0.43428372 0.7538645 ]
Best fitness in Gen 0: 0.7725187289767443
Index of best individual: 1
Best chromosome in Gen 0: [0 0 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 0 1 0 0 1 0 0 0 0]


## 4. Main GA loop over generations

In [9]:
# Track global best after Generation 0
# fitness.max() = best CV-R² found so far
# fitness.argmax() = index of best chromosome in population
# population[best_index_overall].copy() = best chromosome itself
# .copy() to avoid referencing the original array which may change in future generations. Note this for memory management.
# best_history will store the best fitness found in each generation (for plotting later)

best_fitness_overall = fitness.max()
best_index_overall = fitness.argmax()
best_chromosome_overall = population[best_index_overall].copy()

# Keep history of best fitness per generation (including gen 0)
best_history = [best_fitness_overall]

print("Initial best fitness (Gen 0):", best_fitness_overall)

Initial best fitness (Gen 0): 0.7725187289767443


In [12]:
for gen in range(G):
    print(f"\n=== Generation {gen+1} ===")

    # ----------------------------------------
    # 4.1 Compute rank-based selection probabilities
    # ----------------------------------------

    # fitness: array of length P
    # We want ranks: 1 (worst) ... P (best)
    idx_sorted = np.argsort(fitness)  # gives indices sorted by fitness (smallest -> largest)
    ranks = np.zeros(P, dtype=int)

    # Assign ranks according to sorted order
    # idx_sorted[0] -> rank 1 (worst)
    # idx_sorted[-1] -> rank P (best)
    for r, idx_individual in enumerate(idx_sorted, start=1):
        ranks[idx_individual] = r

    # Convert ranks to probabilities
    total_rank = ranks.sum()

    # probability of choosing each individual for parent 1
    # higher fitness -> higher rank -> higher selection probability
    selection_prob = ranks / total_rank  # numpy does elementwise division

    # ----------------------------------------
    # 4.2 Create new population using selection, crossover, mutation
    # ----------------------------------------

   # new_population will store the children of the new generation
    new_population = np.zeros_like(population)  # same shape (P, p)
    child_count = 0 # counts how many children we've already placed

    while child_count < P:

        # Parent 1: chosen by selection_prob
        # parent1_index is sampled using the probabilities in selection_prob, better individuals more likely to be chosen
        parent1_index = np.random.choice(np.arange(P), p=selection_prob)
        parent1 = population[parent1_index]

        # Parent 2: chosen uniformly at random
        # parent2_index is sampled uniformly from 0 to P-1
        parent2_index = np.random.randint(P)
        parent2 = population[parent2_index]

        # Single-point crossover
        # Choose crossover point in [1, p-1]
        crossover_point = np.random.randint(1, p)

        # Create children as copies of parents
        child1 = parent1.copy()
        child2 = parent2.copy()

        # Swap tails after crossover_point
        # child1 takes left side from parent1, right side from parent2
        # child2 takes left side from parent2, right side from parent1
        child1[crossover_point:] = parent2[crossover_point:]
        child2[crossover_point:] = parent1[crossover_point:]

        # Mutation on child1
        # for each position j, we flip the bit with probability mutation_rate. same for child2
        for j in range(p):
            if np.random.rand() < mutation_rate:
                child1[j] = 1 - child1[j]  # flip bit

        # Mutation on child2
        for j in range(p):
            if np.random.rand() < mutation_rate:
                child2[j] = 1 - child2[j]

        # Add children to new_population
        # if P is even, we always add 2 children per iteration
        # if P is odd, we just stop when child_count = P
        new_population[child_count, :] = child1
        child_count += 1

        if child_count < P:
            new_population[child_count, :] = child2
            child_count += 1

    # Replace old population
    population = new_population

    # ----------------------------------------
    # 4.3 Evaluate fitness of new population (same as Gen 0)
    # ----------------------------------------

    fitness = np.zeros(P)

    for i in range(P):
        b = population[i, :]

        if b.sum() == 0:
            fitness[i] = -1e9
            continue

        X_sel = X[:, b == 1]

        SSPE = 0.0

        for train_idx, test_idx in kf.split(X_sel):
            X_train = X_sel[train_idx, :]
            y_train = y[train_idx]
            X_test  = X_sel[test_idx, :]
            y_test  = y[test_idx]

            model = LinearRegression()
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            errors = y_test - y_pred
            SSPE += np.sum(errors ** 2)

        R2_cv = 1.0 - (SSPE / TSS)
        fitness[i] = R2_cv

    # ----------------------------------------
    # 4.4 Update global best and history
    # ----------------------------------------

    # best_fitness_gen = best model in this generation
    # if it beats overall best, update overall best
    best_fitness_gen = fitness.max()
    best_index_gen = fitness.argmax()

    if best_fitness_gen > best_fitness_overall:
        best_fitness_overall = best_fitness_gen
        best_chromosome_overall = population[best_index_gen].copy()

    best_history.append(best_fitness_overall)

    print("Best fitness this generation:", best_fitness_gen)
    print("Best overall fitness so far:", best_fitness_overall)

# ============================================
# 5. After all generations, print final result
# ============================================

print("\n=== FINAL RESULT ===")
print("Best overall fitness (CV-R^2):", best_fitness_overall)
print("Best chromosome (selected predictors):")
print(best_chromosome_overall)
print("Number of predictors selected:",
      best_chromosome_overall.sum())


=== Generation 1 ===
Best fitness this generation: 0.7731465838380933
Best overall fitness so far: 0.7731465838380933

=== Generation 2 ===
Best fitness this generation: 0.7726258461212423
Best overall fitness so far: 0.7731465838380933

=== Generation 3 ===
Best fitness this generation: 0.7742776748869492
Best overall fitness so far: 0.7742776748869492

=== Generation 4 ===
Best fitness this generation: 0.7676743661243033
Best overall fitness so far: 0.7742776748869492

=== Generation 5 ===
Best fitness this generation: 0.7732234613936992
Best overall fitness so far: 0.7742776748869492

=== Generation 6 ===
Best fitness this generation: 0.7749552645138225
Best overall fitness so far: 0.7749552645138225

=== Generation 7 ===
Best fitness this generation: 0.7759533146983529
Best overall fitness so far: 0.7759533146983529

=== Generation 8 ===
Best fitness this generation: 0.7784703820453748
Best overall fitness so far: 0.7784703820453748

=== Generation 9 ===
Best fitness this generati